# Projet de RAG à partir d'un fichier EPUB

In [1]:
import fitz
import os
#import mistralai
#import langchain_mistralai
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_chroma import Chroma
#from langchain_text_splitters import CharacterTextSplitter
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
import json
import shutil
import gc

conf_file = json.load(open(r"C:\Users\jch_m\Nextcloud2\ConfigPerso\api_key.json"))
MISTRAL_API_KEY = conf_file["API_KEY_Mistal"]
#MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")


### RAG sur 1 fichier (IA - Noah Harari)

### RAG sur plusieurs fichiers (recettes)

In [ ]:
files = [
    r"C:\Users\jch_m\Nextcloud2\Lectures\recettesKeto.epub",
    #r"C:\Users\jch_m\Nextcloud2\Lectures\recettesYummy.epub",
    r"C:\Users\jch_m\Nextcloud2\Lectures\180 Recettes Exotiques Faciles.epub",
    r"C:\Users\jch_m\Nextcloud2\Lectures\Jean Imbert 100 recettes éco-responsables.epub",
    r"C:\Users\jch_m\Nextcloud2\Lectures\Recettes minceur à petit prix.epub"
]

text_segment_size = 1000
overlap_size = 150
all_documents = []
banned_keywords = ["All rights reserved", "QUELQUES MERCIS", "Tous droits réservés","Dépôt légal"]

# --- 1. Boucle de traitement des fichiers ---
for path in files:
    if not os.path.exists(path):
        print(f"Fichier introuvable : {path}")
        continue
        
    print(f"Traitement de: {os.path.basename(path)}")
    doc = fitz.open(path)
    
    # --- 2. Découpage intelligent ---
    # On utilise RecursiveCharacterTextSplitter au lieu d'un while manuel
    # car il respecte la structure des phrases et paragraphes.
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=text_segment_size,
        chunk_overlap=overlap_size,
        separators=["\n\n", "\n", ".", " "]
    )

    for page_num in range(len(doc)):
        page = doc[page_num]
        page_text = page.get_text()
        
        if not page_text:
            continue

        if any(word in page_text for word in banned_keywords):
            continue

        # --- EXTRACTION DU TITRE ---
        # On récupère la première ligne de la page comme nom de la recette
        lines = page_text.split('\n')
        recette_nom = lines[0].strip() # La première ligne est le titre

        # On découpe le texte de CETTE page uniquement
        page_chunks = splitter.split_text(page_text)
        
        for chunk in page_chunks:
            all_documents.append(
                Document(
                    page_content=chunk, 
                    metadata={
                        "source": os.path.basename(path), 
                        "page_nb": page_num + 1 , # +1 car l'index commence à 0
                        "recette": recette_nom
                    }
                )
            )
    doc.close()

# --- 3. Vectorisation groupée ---
embeddings = MistralAIEmbeddings(api_key=MISTRAL_API_KEY, model="mistral-embed")

# Supprimer le dossier s'il existe pour repartir de zéro
if os.path.exists("./db_recettes"):
    # On détruit l'objet Python s'il existe
    if 'vectorstore' in locals():
        del vectorstore
    
    # On force la fermeture des fichiers (spécifique à certaines versions de Chroma)
    gc.collect()
    shutil.rmtree("./db_recettes")

# On crée la base avec TOUS les documents des deux livres
vectorstore = Chroma.from_documents(
    documents=all_documents,
    embedding=embeddings,
    persist_directory="./db_recettes"
)


Traitement de: recettesKeto.epub
Traitement de: 180 Recettes Exotiques Faciles.epub
Traitement de: Jean Imbert 100 recettes éco-responsables.epub
MuPDF error: format error: cannot find entry ./fonts/00001.otf

MuPDF error: format error: cannot find entry ./fonts/00002.otf

MuPDF error: format error: cannot find entry ./fonts/00003.otf

MuPDF error: format error: cannot find entry ./fonts/00003.otf

MuPDF error: format error: cannot find entry ./fonts/00004.otf

MuPDF error: format error: cannot find entry ./fonts/00005.otf

MuPDF error: format error: cannot find entry ./fonts/00006.otf

MuPDF error: format error: cannot find entry ./fonts/00007.otf

MuPDF error: format error: cannot find entry ./fonts/00007.otf

MuPDF error: format error: cannot find entry ./fonts/00001.otf

MuPDF error: format error: cannot find entry ./fonts/00002.otf

MuPDF error: format error: cannot find entry ./fonts/00003.otf

MuPDF error: format error: cannot find entry ./fonts/00003.otf

MuPDF error: format er

c:\Dev\RAG_livres\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Questions sur recettes

In [5]:
# --- 4. Test de recherche ---
query = "Quelle recette pour reduire les aigreurs d'estomac?"
results = vectorstore.similarity_search(query, k=3)

print("\n--- Résultats trouvés ---")
for res in results:
    # On affiche la source pour savoir de quel livre vient l'info
    print(f"[{res.metadata['source']}] : page {res.metadata['page_nb']}\n \
Recette: {res.metadata['recette']}\
          \n{res.page_content[:600]}...\n")


--- Résultats trouvés ---
[180 Recettes Exotiques Faciles.epub] : page 164
 Recette: Assaisonner avec du sel et du poivre selon votre goût.          
Assaisonner avec du sel et du poivre selon votre goût....

[180 Recettes Exotiques Faciles.epub] : page 29
 Recette: C'est une explosion de saveurs et de couleurs qui ravira vos          
C'est une explosion de saveurs et de couleurs qui ravira vos
papilles.  
Vous pouvez également personnaliser la recette en ajoutant
d'autres   ingrédients selon vos goûts....

[180 Recettes Exotiques Faciles.epub] : page 243
 Recette: poissons, les plats de riz ou même comme marinade pour donner du          
poissons, les plats de riz ou même comme marinade pour donner du
peps à vos recettes.
N'hésitez pas à ajuster les quantités d'ingrédients selon vos
préférences en matière de piquant et d'arômes....



In [6]:
query = "une recette saine avec du poulet et des légumes"
results = vectorstore.similarity_search(query, k=3)

print("\n--- Résultats trouvés ---")
for res in results:
    # On affiche la source pour savoir de quel livre vient l'info
    print(f"[{res.metadata['source']}] : page {res.metadata['page_nb']}\n \
Recette: {res.metadata['recette']}\
          \n{res.page_content[:600]}...\n")


--- Résultats trouvés ---
[180 Recettes Exotiques Faciles.epub] : page 255
 Recette: N'hésitez pas à personnaliser la recette en ajoutant d'autres          
N'hésitez pas à personnaliser la recette en ajoutant d'autres
légumes ou herbes selon vos préférences....

[Recettes minceur à petit prix.epub] : page 98
 Recette: Poule au pot          
Poule au pot
 4 
 30 min 
 2 h
1 jeune poule d’1,2 à 1,5 kg • 8 carottes nouvelles • 2 gros navets • 4 blancs de
poireaux • 1 cœur de laitue • 1 oignon nouveau • 1 gousse d’ail • 1 bouquet
garni • sel fin • poivre noir du moulin
1 Épluchez tous les légumes et lavez-les à l’eau courante. Placez la poule dans une
marmite et recouvrez d’eau. Portez à ébullition et écumez. 2 Ajoutez les légumes,
l’oignon coupé en deux, le bouquet garni et l’ail écrasé. Salez et poivrez. 3 Couvrez
et laissez frémir à feu moyen pendant 2 bonnes heures. La chair de la poule doit se
détacher. 4 Disposez les légumes au...

[180 Recettes Exotiques Faciles.epub] : page 154
 